<a href="https://colab.research.google.com/github/Fahrizaydr/Epub-Arabic-TTS-Simulation/blob/main/Epub_to_Audio_Facebook_nllb_200_distilled_600M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Library

In [ ]:
!pip install -q transformers datasets ebooklib beautifulsoup4 huggingface-hub

Upload Epub

In [ ]:
from google.colab import files

# Upload file EPUB
uploaded = files.upload()

epub_path = next(iter(uploaded))
print(f"File EPUB yang diupload: {epub_path}")


Baca Epub

In [ ]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re

def extract_text_from_epub(file_path):
    book = epub.read_epub(file_path)
    texts = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content()
            soup = BeautifulSoup(content, 'html.parser')
            texts.append(soup.get_text())

    full_text = "\n".join(texts)
    return full_text

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7Fء-ي]+', ' ', text)
    return text.strip()

# Ekstrak dan bersihkan teks
raw_text = extract_text_from_epub(epub_path)
arabic_text = clean_text(raw_text)

# Preview teks
print("Preview teks Arab:")
print(arabic_text[:1000])


Token Huggingface

In [ ]:
import os
from huggingface_hub import login
import getpass

token = getpass.getpass("Masukkan Hugging Face token kamu: ")
login(token)
os.environ["HF_TOKEN"] = token


Membagi teks pada epub dengan titik sebagai pembatas antar kalimat

In [ ]:
def split_text_by_period(text):
    sentences = text.split('.')
    chunks = []

    for sentence in sentences:
        clean_sentence = sentence.strip()
        if clean_sentence:
            chunks.append(clean_sentence + '.')

    return chunks

# Proses split
chunks = split_text_by_period(arabic_text)

print(f"Total potongan teks (chunk): {len(chunks)}")

try:
    limit = int(input(f"Ingin gunakan berapa chunk pertama? (max {len(chunks)}): "))
    limit = min(limit, len(chunks))
except:
    print("Input tidak valid, gunakan semua chunk.")
    limit = len(chunks)

selected_chunks = chunks[:limit]


Translate Arab > Inggris

In [ ]:
from transformers import pipeline

# Load model NLLB
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

def translate_text(text):
    return translator(text, src_lang="arb", tgt_lang="eng_Latn")[0]['translation_text']

# Terjemahkan
translated_chunks = []
for idx, chunk in enumerate(selected_chunks):
    print(f"Translating chunk {idx+1}/{len(selected_chunks)}")
    try:
        translated = translate_text(chunk)
        translated_chunks.append(translated)
    except Exception as e:
        print(f" Error on chunk {idx+1}: {e}")
        translated_chunks.append("")


Gabungkan hasil translate antar chunk (Inggris)

In [ ]:
# Fungsi menggabungkan chunk hasil terjemahan
def combine_translations(translated_chunks):
    return "\n\n".join(translated_chunks)

full_translation = combine_translations(translated_chunks)

Translate Arab > Indo

In [ ]:
from transformers import pipeline

# Load model NLLB
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

def translate_text(text):
    return translator(text, src_lang="arb", tgt_lang="ind_Latn")[0]['translation_text']

# Terjemahkan
translated_chunks = []
for idx, chunk in enumerate(selected_chunks):
    print(f"Translating chunk {idx+1}/{len(selected_chunks)}")
    try:
        translated = translate_text(chunk)
        translated_chunks.append(translated)
    except Exception as e:
        print(f" Error on chunk {idx+1}: {e}")
        translated_chunks.append("")


Gabungkan hasil translate antar chunk (Indo)

In [ ]:
# Fungsi menggabungkan chunk hasil terjemahan
def combine_translations(translated_chunks):
    return "\n\n".join(translated_chunks)

full_translation = combine_translations(translated_chunks)

Download TXT

In [ ]:
# Simpan ke file txt
output_file = "hasil_terjemahan.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(full_translation)

from google.colab import files
files.download(output_file)

print(f"File hasil terjemahan berhasil disimpan: {output_file}")